In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K

## 1. Load & Prepare Data

In [3]:
# merge features and label
#df= pd.read_csv('25K_tracks_features_and_labels_all.csv')
df= pd.read_csv('../../preprocessing/datasets/ohe_25K_tracks_features_and_labels_all.csv')
df

,Unnamed: 0,track_id,track_genre_top,track_title,artist_name,set_split,set_subset,tempo,rms_harmonic_mean,rms_harmonic_var,...,key_C,key_C#,key_D,key_E,key_Eb,key_F,key_F#,key_G,scale_major,scale_minor
0,0,2,Hip-Hop,Food,AWOL,training,small,161.499023,0.060801,0.000606,...,1,0,0,0,0,0,0,0,0,1
1,1,3,Hip-Hop,Electric Ave,AWOL,training,medium,86.132812,0.068484,0.001239,...,0,0,0,0,0,0,0,1,0,1
2,2,5,Hip-Hop,This World,AWOL,training,small,99.384014,0.079553,0.002230,...,0,1,0,0,0,0,0,0,0,1
3,3,10,Pop,Freeway,Kurt Vile,training,small,112.347147,0.135624,0.000600,...,0,0,0,0,0,0,1,0,1,0
4,4,134,Hip-Hop,Street Music,AWOL,training,medium,135.999178,0.088261,0.001123,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24767,24995,155297,Instrumental,Nebula Reborn,Alex Mason/BlackSunAeon Music,training,medium,123.046875,0.038447,0.000094,...,0,0,1,0,0,0,0,0,1,0
24768,24996,155298,Folk,An Idiot Abroad,Greg Atkinson,training,medium,129.199219,0.139233,0.003167,...,0,0,0,0,0,0,0,1,1,0
24769,24997,155306,Folk,Tiny Man,Greg Atkinson,training,medium,103.359375,0.116513,0.003872,...,0,0,0,0,0,0,0,0,1,0
24770,24998,155307,Experimental,Kolka,AWOTT,training,medium,117.453835,0.085435,0.001713,...,1,0,0,0,0,0,0,0,0,1


In [4]:
#convert categorical variables/target class to code
df_train = df.copy()
# df_train['key'] = LabelEncoder().fit_transform(df_train['key'])
# df_train['scale'] = LabelEncoder().fit_transform(df_train['scale'])
df_train['track_genre_top'] = LabelEncoder().fit_transform(df_train['track_genre_top'])

feature_cols = list(df_train.columns[7:])
print(f'There are {len(feature_cols)} features')
print(feature_cols)
target_col = 'track_genre_top'
classes = df[target_col].unique()
num_class = len(classes)
print(f'There are {num_class} classes, they are:', classes)

There are 88 features
['tempo', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean', 'rms_percussive_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_rolloff_mean', 'spectral_rolloff_var', 'zero_crossings', 'chroma_stft_0_mean', 'chroma_stft_1_mean', 'chroma_stft_2_mean', 'chroma_stft_3_mean', 'chroma_stft_4_mean', 'chroma_stft_5_mean', 'chroma_stft_6_mean', 'chroma_stft_7_mean', 'chroma_stft_8_mean', 'chroma_stft_9_mean', 'chroma_stft_10_mean', 'chroma_stft_11_mean', 'chroma_stft_0_var', 'chroma_stft_1_var', 'chroma_stft_2_var', 'chroma_stft_3_var', 'chroma_stft_4_var', 'chroma_stft_5_var', 'chroma_stft_6_var', 'chroma_stft_7_var', 'chroma_stft_8_var', 'chroma_stft_9_var', 'chroma_stft_10_var', 'chroma_stft_11_var', 'MFCC_1_mean', 'MFCC_2_mean', 'MFCC_3_mean', 'MFCC_4_mean', 'MFCC_5_mean', 'MFCC_6_mean', 'MFCC_7_mean', 'MFCC_8_mean', 'MFCC_9_mean', 'MFCC_10_mean', 'MFCC_11_mean', 'MFCC_12_mean', 'MFCC_13_mean', 'MFCC_14_mean', 'MFCC_15_mean', 'MFCC_16_mean

In [5]:
df[target_col].value_counts()

Rock                   7098
Electronic             6311
Experimental           2250
Hip-Hop                2197
Folk                   1518
Instrumental           1349
Pop                    1186
International          1018
Classical               619
Old-Time / Historic     510
Jazz                    384
Country                 178
Soul-RnB                154
Name: track_genre_top, dtype: int64

In [6]:
#Define train/test/validation & apply scaling
df_training = df_train.loc[df_train['set_split']=='training'].dropna()
df_testing = df_train.loc[df_train['set_split']=='test'].dropna()
df_validation = df_train.loc[df_train['set_split']=='validation'].dropna()

scaler = StandardScaler()

X_train = scaler.fit_transform(df_training[feature_cols])
y_train =  df_training[target_col].values
X_test = scaler.fit_transform(df_testing[feature_cols])
y_test = df_testing[target_col].values
X_val = scaler.fit_transform(df_validation[feature_cols])
y_val = df_validation[target_col].values
print('Training data:',X_train.shape)
print('Testing data:',X_test.shape)
print('Validation data:',X_val.shape)

Training data: (19744, 88)
Testing data: (2546, 88)
Validation data: (2482, 88)


In [7]:
#Oversample minority classes
##In the end we did not use oversampled because it performed worse than un-sampled data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print('Oversampled X_train:', X_train_resampled.shape)

Oversampled X_train: (73801, 88)


## 2. Build Models & Tune Hyperparameters
#### *You can skip this part and go straight to running the final model

In [8]:


# Define hyperparameters to tune
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
hidden_layer_sizes = [[128,64,32], [256,128,64,32], [512,256,128,64,32]]
dropout_rates = [0.2, 0.5]


best_accuracy = 0
best_hyperparameters = None

# Grid search over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
          for dropout_rate in dropout_rates:
              K.clear_session()  # Clear previous model
              model = Sequential()

              for hidden_size in hidden_layer_sizes:
                  for size in hidden_size:
                      model.add(Dense(size, activation='relu'))
                      model.add(Dropout(dropout_rate))
              model.add(Dense(num_class, activation='softmax'))

              optimizer = Adam(learning_rate=lr)
              model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

              # Early stopping to prevent overfitting
              early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

              history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                                    epochs=20, batch_size=batch_size, callbacks=[early_stopping])

              # Evaluate model on validation set
              _, accuracy = model.evaluate(X_val, y_val, verbose=0)

              # Check if this set of hyperparameters improved accuracy
              if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hyperparameters = {'lr': lr, 'batch_size': batch_size, 'hidden_layer_size': hidden_size, 'dropout_rate': dropout_rate}
                    best_model = model
# Print the best hyperparameters
print("Best Hyperparameters:", best_hyperparameters)

Epoch 1/20
617/617 [==============================] - 3s 4ms/step - loss: 2.1978 - accuracy: 0.2576 - val_loss: 1.9895 - val_accuracy: 0.2865
Epoch 2/20
617/617 [==============================] - 2s 4ms/step - loss: 2.0167 - accuracy: 0.3034 - val_loss: 1.8191 - val_accuracy: 0.4436
Epoch 3/20
617/617 [==============================] - 2s 4ms/step - loss: 1.8712 - accuracy: 0.4106 - val_loss: 1.7192 - val_accuracy: 0.4654
Epoch 4/20
617/617 [==============================] - 2s 4ms/step - loss: 1.7995 - accuracy: 0.4345 - val_loss: 1.6641 - val_accuracy: 0.4658
Epoch 5/20
617/617 [==============================] - 2s 4ms/step - loss: 1.7607 - accuracy: 0.4450 - val_loss: 1.6620 - val_accuracy: 0.4718
Epoch 6/20
617/617 [==============================] - 2s 3ms/step - loss: 1.7282 - accuracy: 0.4499 - val_loss: 1.6591 - val_accuracy: 0.4980
Epoch 7/20
617/617 [==============================] - 2s 4ms/step - loss: 1.7035 - accuracy: 0.4587 - val_loss: 1.6573 - val_accuracy: 0.4984
Epoch 

Epoch 1/20
617/617 [==============================] - 3s 3ms/step - loss: 2.5056 - accuracy: 0.1720 - val_loss: 2.4107 - val_accuracy: 0.2865
Epoch 2/20
617/617 [==============================] - 2s 3ms/step - loss: 2.2633 - accuracy: 0.2606 - val_loss: 2.2360 - val_accuracy: 0.2865
Epoch 3/20
617/617 [==============================] - 2s 4ms/step - loss: 2.1714 - accuracy: 0.2677 - val_loss: 2.1604 - val_accuracy: 0.2865
Epoch 4/20
617/617 [==============================] - 2s 3ms/step - loss: 2.1239 - accuracy: 0.2737 - val_loss: 2.1362 - val_accuracy: 0.2865
Epoch 5/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0995 - accuracy: 0.2820 - val_loss: 2.1337 - val_accuracy: 0.2865
Epoch 6/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0721 - accuracy: 0.2865 - val_loss: 2.1228 - val_accuracy: 0.2865
Epoch 7/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0582 - accuracy: 0.2878 - val_loss: 2.1238 - val_accuracy: 0.2865
Epoch 

Epoch 1/20
309/309 [==============================] - 2s 4ms/step - loss: 2.2957 - accuracy: 0.2409 - val_loss: 2.1158 - val_accuracy: 0.2881
Epoch 2/20
309/309 [==============================] - 1s 4ms/step - loss: 2.0935 - accuracy: 0.2770 - val_loss: 1.9855 - val_accuracy: 0.3904
Epoch 3/20
309/309 [==============================] - 1s 4ms/step - loss: 1.9691 - accuracy: 0.3706 - val_loss: 1.7420 - val_accuracy: 0.4645
Epoch 4/20
309/309 [==============================] - 1s 4ms/step - loss: 1.8723 - accuracy: 0.4244 - val_loss: 1.7251 - val_accuracy: 0.4662
Epoch 5/20
309/309 [==============================] - 1s 4ms/step - loss: 1.8218 - accuracy: 0.4388 - val_loss: 1.7134 - val_accuracy: 0.4718
Epoch 6/20
309/309 [==============================] - 1s 5ms/step - loss: 1.7898 - accuracy: 0.4468 - val_loss: 1.7096 - val_accuracy: 0.4726
Epoch 7/20
309/309 [==============================] - 1s 4ms/step - loss: 1.7638 - accuracy: 0.4513 - val_loss: 1.6845 - val_accuracy: 0.4730
Epoch 

Epoch 1/20
309/309 [==============================] - 2s 4ms/step - loss: 2.5468 - accuracy: 0.1569 - val_loss: 2.4857 - val_accuracy: 0.2542
Epoch 2/20
309/309 [==============================] - 1s 4ms/step - loss: 2.3597 - accuracy: 0.2488 - val_loss: 2.3458 - val_accuracy: 0.2542
Epoch 3/20
309/309 [==============================] - 1s 4ms/step - loss: 2.2435 - accuracy: 0.2580 - val_loss: 2.2442 - val_accuracy: 0.2865
Epoch 4/20
309/309 [==============================] - 1s 4ms/step - loss: 2.1839 - accuracy: 0.2689 - val_loss: 2.1885 - val_accuracy: 0.2865
Epoch 5/20
309/309 [==============================] - 1s 4ms/step - loss: 2.1506 - accuracy: 0.2701 - val_loss: 2.1624 - val_accuracy: 0.2865
Epoch 6/20
309/309 [==============================] - 1s 4ms/step - loss: 2.1182 - accuracy: 0.2745 - val_loss: 2.1457 - val_accuracy: 0.2865
Epoch 7/20
309/309 [==============================] - 1s 4ms/step - loss: 2.0974 - accuracy: 0.2720 - val_loss: 2.1428 - val_accuracy: 0.2865
Epoch 

Epoch 1/20
155/155 [==============================] - 2s 6ms/step - loss: 2.3156 - accuracy: 0.2270 - val_loss: 2.1009 - val_accuracy: 0.2865
Epoch 2/20
155/155 [==============================] - 1s 6ms/step - loss: 2.1038 - accuracy: 0.2651 - val_loss: 2.0257 - val_accuracy: 0.2865
Epoch 3/20
155/155 [==============================] - 1s 6ms/step - loss: 2.0620 - accuracy: 0.2705 - val_loss: 2.0115 - val_accuracy: 0.2865
Epoch 4/20
155/155 [==============================] - 1s 6ms/step - loss: 2.0309 - accuracy: 0.2800 - val_loss: 1.9526 - val_accuracy: 0.4025
Epoch 5/20
155/155 [==============================] - 1s 6ms/step - loss: 1.9732 - accuracy: 0.3284 - val_loss: 1.7816 - val_accuracy: 0.4537
Epoch 6/20
155/155 [==============================] - 1s 6ms/step - loss: 1.8851 - accuracy: 0.3991 - val_loss: 1.7284 - val_accuracy: 0.4517
Epoch 7/20
155/155 [==============================] - 1s 6ms/step - loss: 1.8366 - accuracy: 0.4163 - val_loss: 1.7048 - val_accuracy: 0.4529
Epoch 

...

Epoch 1/20
155/155 [==============================] - 2s 6ms/step - loss: 2.0292 - accuracy: 0.3113 - val_loss: 1.7223 - val_accuracy: 0.4658
Epoch 2/20
155/155 [==============================] - 1s 6ms/step - loss: 1.7761 - accuracy: 0.4491 - val_loss: 1.6133 - val_accuracy: 0.5040
Epoch 3/20
155/155 [==============================] - 1s 6ms/step - loss: 1.6856 - accuracy: 0.4743 - val_loss: 1.5547 - val_accuracy: 0.5081
Epoch 4/20
155/155 [==============================] - 1s 6ms/step - loss: 1.6432 - accuracy: 0.4759 - val_loss: 1.5668 - val_accuracy: 0.5214
Epoch 5/20
155/155 [==============================] - 1s 6ms/step - loss: 1.6025 - accuracy: 0.4873 - val_loss: 1.5250 - val_accuracy: 0.5274
Epoch 6/20
155/155 [==============================] - 1s 6ms/step - loss: 1.5681 - accuracy: 0.4994 - val_loss: 1.4893 - val_accuracy: 0.5342
Epoch 7/20
155/155 [==============================] - 1s 6ms/step - loss: 1.5499 - accuracy: 0.5037 - val_loss: 1.4479 - val_accuracy: 0.5411
Epoch 

Epoch 1/20
155/155 [==============================] - 2s 6ms/step - loss: 2.2970 - accuracy: 0.2415 - val_loss: 2.1574 - val_accuracy: 0.2865
Epoch 2/20
155/155 [==============================] - 1s 6ms/step - loss: 2.0947 - accuracy: 0.2747 - val_loss: 2.1208 - val_accuracy: 0.2865
Epoch 3/20
155/155 [==============================] - 1s 6ms/step - loss: 2.0398 - accuracy: 0.2876 - val_loss: 2.1219 - val_accuracy: 0.2865
Epoch 4/20
155/155 [==============================] - 1s 6ms/step - loss: 2.0081 - accuracy: 0.3207 - val_loss: 2.1089 - val_accuracy: 0.2579
Epoch 5/20
155/155 [==============================] - 1s 6ms/step - loss: 1.9725 - accuracy: 0.3620 - val_loss: 2.0849 - val_accuracy: 0.2542
Epoch 6/20
155/155 [==============================] - 1s 6ms/step - loss: 1.9522 - accuracy: 0.3874 - val_loss: 2.1012 - val_accuracy: 0.2542
Epoch 7/20
155/155 [==============================] - 1s 6ms/step - loss: 1.9279 - accuracy: 0.4127 - val_loss: 2.0676 - val_accuracy: 0.2542
Epoch 

Epoch 1/20
617/617 [==============================] - 3s 4ms/step - loss: 2.0704 - accuracy: 0.2840 - val_loss: 2.0553 - val_accuracy: 0.2865
Epoch 2/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0912 - accuracy: 0.2851 - val_loss: 2.0547 - val_accuracy: 0.2865
Epoch 3/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0461 - accuracy: 0.2875 - val_loss: 2.0546 - val_accuracy: 0.2865
Epoch 4/20
617/617 [==============================] - 2s 4ms/step - loss: 2.0476 - accuracy: 0.2875 - val_loss: 2.0544 - val_accuracy: 0.2865
Epoch 5/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0444 - accuracy: 0.2875 - val_loss: 2.0535 - val_accuracy: 0.2865
Epoch 6/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0445 - accuracy: 0.2875 - val_loss: 2.0549 - val_accuracy: 0.2865
Epoch 7/20
617/617 [==============================] - 2s 3ms/step - loss: 2.0444 - accuracy: 0.2844 - val_loss: 2.0539 - val_accuracy: 0.2865
Epoch 

Epoch 1/20
 69/617 [==>...........................] - ETA: 1s - loss: 2.2372 - accuracy: 0.2545

InvalidArgumentError: Graph execution error:

Detected at node Adam/truediv_15 defined at (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Library/Frameworks/Python.framework/Ver

In [9]:
best_hyperparameters

{'lr': 0.001,
 'batch_size': 128,
 'hidden_layer_size': [512, 256, 128, 64, 32],
 'dropout_rate': 0.2}

In [10]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy)

20/20 [==============================] - 0s 2ms/step - loss: 1.4869 - accuracy: 0.5416
The test loss is : 1.4869064092636108

The test Accuracy is : 0.5416339635848999


## 3. Retrain final model using the best model architecture & hyperparameters

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(256,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(num_class, activation="softmax"),

])
print(model.summary())
model.compile(optimizer=Adam(lr=0.001),
                  loss='sparse_categorical_crossentropy',
                   metrics='accuracy')

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30,
                     batch_size=128)

test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 512)               45568     
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 256)               131328    
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_15 (Dense)            (None, 128)               32896     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)               

None
Epoch 1/30
155/155 [==============================] - 1s 4ms/step - loss: 1.7524 - accuracy: 0.4481 - val_loss: 1.3834 - val_accuracy: 0.5834
Epoch 2/30
155/155 [==============================] - 1s 4ms/step - loss: 1.4748 - accuracy: 0.5444 - val_loss: 1.3287 - val_accuracy: 0.5999
Epoch 3/30
155/155 [==============================] - 1s 4ms/step - loss: 1.3911 - accuracy: 0.5760 - val_loss: 1.2917 - val_accuracy: 0.6064
Epoch 4/30
155/155 [==============================] - 1s 4ms/step - loss: 1.3386 - accuracy: 0.5882 - val_loss: 1.2571 - val_accuracy: 0.6160
Epoch 5/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2957 - accuracy: 0.6032 - val_loss: 1.2519 - val_accuracy: 0.6172
Epoch 6/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2607 - accuracy: 0.6128 - val_loss: 1.2424 - val_accuracy: 0.6176
Epoch 7/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2221 - accuracy: 0.6259 - val_loss: 1.2469 - val_accuracy: 0.6116
E

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Use argmax for categorical predictions


# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

# Print precision
precision = precision_score(y_test, y_pred_classes, average='weighted')
print("Precision:", precision)

# Print recall
recall = recall_score(y_test, y_pred_classes, average='weighted')
print("Recall:", recall)

# Print F1 score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("F1 Score:", f1)

80/80 [==============================] - 0s 713us/step
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.81      0.77        62
           1       0.00      0.00      0.00        18
           2       0.64      0.71      0.67       632
           3       0.33      0.36      0.34       225
           4       0.22      0.28      0.24       152
           5       0.68      0.65      0.66       220
           6       0.37      0.22      0.28       174
           7       0.48      0.28      0.36       102
           8       0.56      0.36      0.44        39
           9       0.91      0.98      0.94        51
          10       0.14      0.08      0.10       119
          11       0.68      0.78      0.72       710
          12       0.00      0.00      0.00        42

    accuracy                           0.57      2546
   macro avg       0.44      0.42      0.43      2546
weighted avg       0.54      0.57      0.55      2546

C

/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

In [13]:
confusion_matrix(y_test, y_pred_classes)

array([[ 50,   0,   1,   4,   0,   0,   6,   0,   1,   0,   0,   0,   0],
       [  0,   0,   4,   0,   0,   1,   0,   0,   0,   0,   2,  11,   0],
       [  5,   0, 449,  45,  16,  29,  15,   9,   5,   1,  10,  48,   0],
       [  4,   0,  48,  80,  18,   5,  16,   1,   0,   0,   6,  47,   0],
       [  0,   0,  11,  20,  42,   1,   6,   7,   2,   3,  10,  50,   0],
       [  0,   0,  54,   9,   1, 142,   1,   6,   0,   0,   2,   5,   0],
       [  4,   0,  21,  27,  53,   1,  38,   2,   0,   0,   8,  20,   0],
       [  0,   0,  15,   2,  24,   5,   6,  29,   0,   0,   2,  19,   0],
       [  2,   0,   4,   2,   6,   0,   7,   0,  14,   0,   1,   3,   0],
       [  0,   0,   0,   1,   0,   0,   0,   0,   0,  50,   0,   0,   0],
       [  0,   0,  43,   5,  13,  10,   2,   1,   0,   0,  10,  35,   0],
       [  3,   0,  41,  46,  20,  13,   5,   4,   3,   1,  22, 552,   0],
       [  0,   0,  12,   0,   0,   2,   0,   2,   0,   0,   1,  25,   0]])

In [19]:
model.save('cnn_genre_prediction.h5')

/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [105]:
type(X_train)

numpy.ndarray

In [57]:
# to get feature importances

import shap

background = X_train[np.random.choice(X_train.shape[0], 10000, replace=False)]

# Create a SHAP Explainer
explainer = shap.Explainer(model, background)

# Select a sample (or multiple samples) for explanation
sample = X_test[np.random.choice(X_test.shape[0], 1000, replace=False)]

# Calculate SHAP values
shap_values = explainer(sample)


PermutationExplainer explainer: 1001it [01:25, 10.59it/s]                         


In [88]:
feature_importances_per_class = {}

# Iterate over each class (label)
for label_index in range(shap_values.shape[2]):
    # Calculate mean absolute SHAP values for each feature for this label
    mean_abs_shap_per_feature = np.abs(shap_values.values[:, :, label_index]).mean(axis=0)
    
    # Assuming 'feature_cols' is a list of your feature names
    feature_importances_per_class[f'Label_{label_index}'] = dict(zip(feature_cols, mean_abs_shap_per_feature))

# Convert the dictionary to a DataFrame for easier handling
df_feature_importances = pd.DataFrame(feature_importances_per_class)



In [99]:
cnn_fi_df = pd.DataFrame(df_feature_importances.T.mean()).T

In [101]:
cnn_fi_df.to_csv('cnn_feature_importances.csv')